In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
import cv2
from time import time
from sklearn.decomposition import IncrementalPCA

In [2]:
def read_video(filename, nframes=np.inf):
    """
        Read the given number of frames of a video using 
        the opencv library
    """
    frames = []
    cap = cv2.VideoCapture(filename)
    fps = round(cap.get(cv2.CAP_PROP_FPS))
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    i = 0
    while cap.isOpened() and i < nframes:
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
            i += 1
        else:
            break
    
    cap.release()
    return np.array(frames), fps, width, height

In [3]:
print('Video loading...')
# Read the video
start = time()
sample_name = 'sample20s_540'
frames, fps, width, height = read_video('data/'+sample_name+'.mp4')
nframes = frames.shape[0]
print(f'Loaded video of {round(nframes/fps, 2)} seconds with quality {width}x{height} in {round(time()-start,2)} seconds')

# Convert to gray scale
print('Transforming the video...')
gray_frames = np.zeros((nframes, height, width), dtype=np.uint8)
for i, frame in enumerate(frames):
    gray_frames[i] = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Create, fit and apply the pca model
pca_model = IncrementalPCA(n_components=50)
start = time()
pca_model.fit(gray_frames.reshape(gray_frames.shape[0], -1))
after_fit = time()
reduced_gray_frames = pca_model.transform(gray_frames.reshape(gray_frames.shape[0], -1))
gray_frames_2 = pca_model.inverse_transform(reduced_gray_frames).reshape((nframes, height, width))
after_transformations = time()
print(f'Fitting of the pca model took {round(after_fit-start,2)} seconds')
print(f'Transformations of the videos took {round(after_transformations-after_fit,2)} seconds')

# Write the videos to disk
writer_after_pca = cv2.VideoWriter('data/'+sample_name+'_after_pca.mp4', cv2.VideoWriter_fourcc(*"MJPG"),
                         fps, (width, height))
writer_gray = cv2.VideoWriter('data/'+sample_name+'_gray.mp4', cv2.VideoWriter_fourcc(*"MJPG"),
                         fps, (width, height), 0)
writer_gray_after_pca = cv2.VideoWriter('data/'+sample_name+'_gray_after_pca.mp4', cv2.VideoWriter_fourcc(*"MJPG"),
                         fps, (width, height), 0)
print('Displaying and writting the video...')
for i in range(nframes):
    frame_pca = np.around(gray_frames_2[i]).astype(np.uint8)
    writer_gray.write(gray_frames[i])
    writer_gray_after_pca.write(frame_pca)
    
    cv2.imshow('Video Capture', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    
# When everything done, release the video capture and video write objects
writer_after_pca.release()
writer_gray.release()
writer_gray_after_pca.release()

# Closes all the frames
cv2.destroyAllWindows()
print('Done')

Video loading...
Loaded video of 20.0 seconds with quality 960x540 in 1.13 seconds
Transforming the video...
Fitting of the pca model took 124.42 seconds
Transformations of the videos took 2.3 seconds
Displaying and writting the video...


In [16]:
import os
import subprocess
import linecache
import csv
from torch.utils.data import Dataset, DataLoader

"""
    Code taken from https://discuss.pytorch.org/t/dataloaders-multiple-files-and-multiple-rows-per-column-with-lazy-evaluation/11769/8
"""

class LazyTextDataset(Dataset):
    def __init__(self, filename):
        self._filename = filename
        self._total_data = int(subprocess.check_output("wc -l " + filename, shell=True).split()[0])

    def __getitem__(self, idx):
        line = linecache.getline(self._filename, idx + 1)
        csv_line = csv.reader([line])
        return next(csv_line)
      
    def __len__(self):
        return self._total_data

path = 'data/test/'
files = list(map(lambda x : path + x, (filter(lambda x : x.endswith("csv"), os.listdir(path)))))
datasets = list(map(lambda x : LazyTextDataset(x), files))

In [4]:
from video_loader import video_loader
from sklearn.decomposition import IncrementalPCA

videos = !ls data/sample*.mp4
videos1 = videos[0:1]
loader = video_loader(videos1, 20, create_batch=False)
frames = loader.create_frames()
frames = frames[0].unsqueeze(1).transpose(1, 4).squeeze()

#frames = frames[:, :, :72, :128] #TEMP
frames = frames.reshape(frames.shape[0], -1)

batch = frames.split(100, 0)
ncomp = 10
pca_model1 = IncrementalPCA(n_components=ncomp)
pca_model2 = IncrementalPCA(n_components=ncomp)

for b in batch:
    pca_model1.partial_fit(b)

In [2]:
from sklearn.decomposition import IncrementalPCA
from torchvision.io import read_video

# NFRAMES x WIDTH x HEIGHT x CHANNElS
ncomp = 10
filename = 'data/sample1.mp4'
pca_model = IncrementalPCA(n_components=ncomp)
for start in range(0, 16, 5):
    vframes = read_video(filename, start_pts=start, end_pts=start+5, pts_unit='sec')[0]
    print(vframes.shape)
    vframes = vframes.reshape(vframes.shape[0], -1)
    pca_model.partial_fit(vframes)

torch.Size([120, 720, 1280, 3])
torch.Size([121, 720, 1280, 3])
torch.Size([121, 720, 1280, 3])
torch.Size([121, 720, 1280, 3])


In [41]:
partial_result = pca_model1.transform(frames)

In [35]:
full_result = pca_model2.fit_transform(frames)